In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rectal_Cancer"
cohort = "GSE109057"

# Input paths
in_trait_dir = "../../input/GEO/Rectal_Cancer"
in_cohort_dir = "../../input/GEO/Rectal_Cancer/GSE109057"

# Output paths
out_data_file = "../../output/preprocess/Rectal_Cancer/GSE109057.csv"
out_gene_data_file = "../../output/preprocess/Rectal_Cancer/gene_data/GSE109057.csv"
out_clinical_data_file = "../../output/preprocess/Rectal_Cancer/clinical_data/GSE109057.csv"
json_path = "../../output/preprocess/Rectal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome analysis by expression microarrays of rectal cancer derived from the Cancer Institute Hospital of Japanese Foundation for Cancer Research"
!Series_summary	"We performed the expression microrarray experiments for mRNA of rectal cancer tissue."
!Series_overall_design	"Messenger RNA were extracted from primary tumor of 90 rectal cancer patients, hybridized and scanned with Affymetrix PrimeView Human Gene Expression Array. There are 91 samples because one sample was hybridized twice, once in each batch, and is represented by GSM2928780 and GSM2928794."
Sample Characteristics Dictionary:
{0: ['tissue: rectal cancer'], 1: ['Sex: M', 'Sex: F'], 2: ['age: 35 <= age < 40', 'age: 30 <= age < 35', 'age: 55 <= age < 60', 'age: 45 <= age < 50', 'age: 65 <= age < 70', 'age: 75 <= age < 80', 'age: 70 <= age < 75', 'age: 60 <= age < 65', 'age: 50 <= age < 55', 'age: 80 <= age < 85', 'age: 40 <= age < 45'], 3: ['batch: Batch 1', 'batch: Batch 2']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on background information, this dataset contains gene expression data from Affymetrix arrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Rectal Cancer is the trait we're studying, and it's available in row 0
trait_row = 0

# Gender data is available in row 1
gender_row = 1

# Age data is available in row 2
age_row = 2

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    # All samples are rectal cancer "tissue: rectal cancer"
    if not isinstance(value, str):
        return None
    
    # Extract the value after colon
    parts = value.split(": ")
    if len(parts) < 2:
        return None
    
    tissue = parts[1].strip()
    if "rectal cancer" in tissue.lower():
        return 1
    else:
        return 0

def convert_gender(value):
    # Gender data in format "Sex: M" or "Sex: F"
    if not isinstance(value, str):
        return None
    
    # Extract the value after colon
    parts = value.split(": ")
    if len(parts) < 2:
        return None
    
    gender = parts[1].strip()
    if gender == 'M':
        return 1
    elif gender == 'F':
        return 0
    else:
        return None

def convert_age(value):
    # Age data in format "age: X <= age < Y"
    if not isinstance(value, str):
        return None
    
    # Extract the value after colon
    parts = value.split(": ")
    if len(parts) < 2:
        return None
    
    age_range = parts[1].strip()
    # Extract the lower and upper bounds of the age range
    try:
        age_parts = age_range.split(" ")
        lower_bound = int(age_parts[0])
        upper_bound = int(age_parts[-1])
        # Return the midpoint of the age range
        return (lower_bound + upper_bound) / 2
    except:
        return None

# 3. Save Metadata
# trait_row is not None, so is_trait_available should be True
is_trait_available = trait_row is not None

validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview and save clinical data
    print("Clinical Data Preview:")
    print(preview_df(clinical_df))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical data to CSV
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM2928730': [1.0, 37.5, 1.0], 'GSM2928731': [1.0, 32.5, 1.0], 'GSM2928732': [1.0, 57.5, 1.0], 'GSM2928733': [1.0, 47.5, 1.0], 'GSM2928734': [1.0, 67.5, 0.0], 'GSM2928735': [1.0, 77.5, 1.0], 'GSM2928736': [1.0, 57.5, 1.0], 'GSM2928737': [1.0, 47.5, 1.0], 'GSM2928738': [1.0, 72.5, 1.0], 'GSM2928739': [1.0, 62.5, 1.0], 'GSM2928740': [1.0, 52.5, 1.0], 'GSM2928741': [1.0, 32.5, 1.0], 'GSM2928742': [1.0, 67.5, 1.0], 'GSM2928743': [1.0, 47.5, 1.0], 'GSM2928744': [1.0, 67.5, 1.0], 'GSM2928745': [1.0, 57.5, 1.0], 'GSM2928746': [1.0, 47.5, 1.0], 'GSM2928747': [1.0, 47.5, 1.0], 'GSM2928748': [1.0, 72.5, 1.0], 'GSM2928749': [1.0, 67.5, 1.0], 'GSM2928750': [1.0, 47.5, 1.0], 'GSM2928751': [1.0, 67.5, 1.0], 'GSM2928752': [1.0, 67.5, 0.0], 'GSM2928753': [1.0, 57.5, 1.0], 'GSM2928754': [1.0, 52.5, 1.0], 'GSM2928755': [1.0, 67.5, 1.0], 'GSM2928756': [1.0, 47.5, 1.0], 'GSM2928757': [1.0, 62.5, 1.0], 'GSM2928758': [1.0, 62.5, 1.0], 'GSM2928759': [1.0, 67.5, 0.0], 'GSM2928760': [1

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print(gene_data.index[:20])


Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These appear to be probe set IDs from an Affymetrix microarray platform, not human gene symbols.
# The format "11715100_at" is typical of Affymetrix probe identifiers.
# These will need to be mapped to standard human gene symbols for analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the preview, it looks like:
# - 'ID' column in gene_annotation contains the same probe IDs as seen in the gene expression data
# - 'Gene Symbol' column contains the gene symbols we want to map to

# 2. Get a gene mapping dataframe by extracting these two columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Print a preview of the mapping data for verification
print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Convert probe-level measurements to gene expression data by applying the gene mapping
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first 10 gene symbols to verify the conversion
print("\nGene data preview (first 10 genes):")
print(gene_data.index[:10])

# Print the shape of the converted dataset
print(f"\nGene data shape: {gene_data.shape}")


Gene mapping preview (first 5 rows):
              ID       Gene
0    11715100_at   HIST1H3G
1  11715101_s_at   HIST1H3G
2  11715102_x_at   HIST1H3G
3  11715103_x_at  TNFAIP8L1
4  11715104_s_at      OTOP2



Gene data preview (first 10 genes):
Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS',
       'AACS', 'AACSP1'],
      dtype='object', name='Gene')

Gene data shape: (19963, 91)


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Get clinical data from previous step
# Load the clinical features from the saved file
clinical_file_path = out_clinical_data_file
if os.path.exists(clinical_file_path):
    clinical_features = pd.read_csv(clinical_file_path)
    print(f"Clinical features loaded from {clinical_file_path}")
    print(f"Clinical features shape: {clinical_features.shape}")
else:
    # If file doesn't exist, we need to extract it again
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=2,
        convert_trait=convert_trait,
        age_row=3,
        convert_age=convert_age,
        gender_row=None,
        convert_gender=None
    )
    print(f"Clinical features re-extracted")
    print(f"Clinical features shape: {clinical_features.shape}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features.T, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(f"First few columns: {list(linked_data.columns[:5])}")

# Check what columns are available in the linked data
print(f"Columns in linked_data: {list(linked_data.columns[:5])}")

# 3. Handle missing values in the linked data
# Find the correct trait column name (it should be the first column)
trait_column = linked_data.columns[0]
print(f"Using trait column: {trait_column}")

linked_data_processed = handle_missing_values(linked_data, trait_column)
print(f"Shape after handling missing values: {linked_data_processed.shape}")

# Add validation check - if no samples remain, note the issue
if linked_data_processed.shape[0] == 0:
    print("No samples remain after handling missing values. The dataset cannot be processed further.")
    is_trait_biased = True  # Mark as biased since we can't use it
    unbiased_linked_data = linked_data_processed
else:
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_processed, trait_column)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from rectal cancer patients with treatment response data (sensitive/resistant)."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data shape: (19758, 91)
First few normalized gene symbols: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AACSP1']


Normalized gene data saved to ../../output/preprocess/Rectal_Cancer/gene_data/GSE109057.csv
Clinical features loaded from ../../output/preprocess/Rectal_Cancer/clinical_data/GSE109057.csv
Clinical features shape: (3, 91)
Linked data shape: (94, 19849)
First few columns: ['GSM2928730', 'GSM2928731', 'GSM2928732', 'GSM2928733', 'GSM2928734']
Columns in linked_data: ['GSM2928730', 'GSM2928731', 'GSM2928732', 'GSM2928733', 'GSM2928734']
Using trait column: GSM2928730
Shape after handling missing values: (3, 91)
For the feature 'GSM2928730', the least common label is '37.5' with 1 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'GSM2928730' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Rectal_Cancer/cohort_info.json
Linked data saved to ../../output/preprocess/Rectal_Cancer/GSE109057.csv
